## Kaggle Netflix - Movie recommendation
https://www.kaggle.com/laowingkin/netflix-movie-recommendation

### 라이브러리 호출

In [18]:
! pip install surprise

In [ ]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
sns.set_style("darkgrid")

### 데이터 불러오기

In [ ]:
# csv파일 읽어오기
# header = None : read_csv의 header default value는 header = 0이므로 column name이 없으면 0번 행의 값이 
# 헤더로 쓰이게 된다. 따라서 header = None으로 header가 없음을 명시해주어야 한다.
# names = ['Cust_Id', 'Rating'] : header = None으로 헤더가 없음을 명시했으므로 헤더 자리에 들어갈 column 이름을
# 지정해주어야 한다.
# use_cols = [0, 1] : 데이터 셋에는 Movie ID, Custormer ID, Rating, Date they gave the ratings column이 있는데,
# 여기서 Date를 제외한 'Cust_Id', 'Rating'만 가져와서 각각 이름을 지정해준 것이다.
df1 = pd.read_csv("../input/combined_data_1.txt", header = None, names = ['Cust_Id', 'Rating'], usecols = [0, 1])

# 'Rating' column의 데이터 형식을 float형식으로 바꿔준다. 
df1['Rating'] = df1['Rating'].astype('float')

print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')
# 행은 5000000으로 나눠지는 모든 행, 열은 모든 열을 출력한다.
# ::a : a로 나누어 떨어지는 번 째에 있는 행들. 물론, 0번째 행도 포함한다.
print(df1.iloc[::5000000, :])

# 출력 값
# Dataset 1 shape: (24058263, 2)
# -Dataset examples-
#           Cust_Id  Rating
# 0              1:     NaN
# 5000000   2560324     4.0
# 10000000  2271935     2.0
# 15000000  1921803     2.0
# 20000000  1933327     3.0

### Pandas practice
- .iloc[행, 열]
- .iloc[a] : a에 해당하는 모든 행
- .iloc[a, b] : a와 b에 해당하는 모든 데이터
- ex) .iloc[:2, :3] : 0~1행, 0~2열에 해당하는 모든 값들. 즉, 2 by 3 행렬이 선택된다.

- groupby('a')['a'].agg(['count']) : a에 대하여 groupby한 뒤 a열의 값을들 count하여 count열에 넣음

In [90]:
mydict = [{'a': np.nan, 'b': 3, 'c': 6, 'd': 9},
          {'a': 1, 'b': 4, 'c': 7, 'd': 10},
          {'a': 2, 'b': 5, 'c': 8, 'd': 11}]
my_df = pd.DataFrame(mydict)
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

c = my_df.groupby('a')['a'].agg(['count'])
print(my_df)
print(my_df.isnull().sum())
print(c.sum())
print(my_df.iloc[-1,0])
string = '1234569'
my_df = my_df[pd.notnull(my_df['a'])]
print(my_df)

     a  b  c   d
0  NaN  3  6   9
1  1.0  4  7  10
2  2.0  5  8  11
a    1
b    0
c    0
d    0
dtype: int64
count    2
dtype: int64
2.0
     a  b  c   d
1  1.0  4  7  10
2  2.0  5  8  11


### Combine Dataset
- 원래는 총 4개의 csv파일이 있기 때문에 이를 통합해 주어야 한다.

In [ ]:
df = df1
# df = df.append(df2)
# df = df.append(df3)
# df = df.append(df4)

# 데이터들을 합쳤으므로, 데이터의 index에 번호를 부여한다.
df.index = np.arrange(0, len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

### Data Viewing

In [ ]:
# df를 'Rating' 컬럼을 기준으로 groupby 시키면 각 값들(ex. 4.0, 3.5 .etc)에는 여러 Cust_Id가 있을텐데,
# 이를 'Rating' 열에 대해 .agg(['count'])를 적용시켜주면 Index가 rating이고 count열이 새로 생겨서 count열의 값은
# 각 Rating의 값들을 count한 값이 들어간다.
# 위 Pandas practice 참고
p = df.groupby('Rating')['Rating'].agg(['count'])

# get movie count : df의 결측치를 찾은 뒤 레이팅 값의 결측치의 개수를 movie_count에 넣는다.
movie_count = df.isnull().sum()[1]

# get customer count : df의 'Cust_Id'열에 unique한 값의 개수에서 결측치의 개수를 뺀다.
cust_count = df['Cust_Id'].nunique() - movie_count

# get rating count : df의 'Cust_Id'열의 개수에서 결측치의 개수를 뺀다.
rating_count = df['Cust_Id'].count() - movie_count

# p에 대해 barh그래프를 그리고, legend는 없애고, 그래프 크기는 (15,10)으로 한다.
# plot의 barh는 horizontal bar plot으로, 수평적인 막대그래프이다.
# legend = False : legend는 범례로, 예를들어 남녀 성별 그래프를 그릴 때 보통 그래프 오른쪽에 있는 그래프 색깔 별 성별을
# 나타내는 표시이다.
ax = p.plot(kind = 'barh', legend = False, figsize = (15,10))
#plt.의 
plt.title('Total pool: {:,} Movies, {:,} customers, {:,} ratings given'.format(movie_count, cust_count, rating_count)
, fontsize=20)
plt.axis('off')

# Rating의 값은 1.0, 2.0, 3.0, 4.0, 5.0이 있기 때문에 range(1,6)으로 1부터 5까지 반복해준다.
for i in range(1,6):
    ax.text(p.iloc[i-1][0]/4, i-1, 'Rating {}: {:.0f}%'.format(i, p.iloc[i-1][0]*100 / p.sum()[0]), 
    color = 'white', weight = 'bold')


### Data cleaning
Movie ID is really a mess import! Looping through dataframe to add Movie ID column WILL make the Kernel run out of memory as it is too inefficient. I achieve my task by first creating a numpy array with correct length then add the whole array as column into the main dataframe! Let's see how it is done below:

Movie ID 열은 매우 복잡하다! Movie ID열을 데이터프레임에 추가하기 위해서 데이터프레임을 반복하는 것은 커널 메모리가 부족해서 매우 비효율적이다. 먼저 데이터프레임에 맞는 NumPy 배열을 만든 뒤 전체 열을 데이터프레임에 column으로 추가하였다.
어떻게 했는지 아래에서 확인해보자:

In [ ]:
# df의 Rating열의 결측치를 확인하는 isnull메서드의 결과 df를 df_nan이란 데이터프레임에 넣는다.
df_nan = pd.DataFrame(pd.isnull(df.Rating))
# df_nan의 'Rating' 컬럼에서 True인 값들만 df_nan에 넣는다. 이때 df_nan의 모습은 'Rating'열의 값이 True인 행들만 
# 모여있다.
df_nan = df_nan[df_nan['Rating'] == True]
# df_nan의 인덱스를 제거한다.
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

# 예를 들어 결측치 인덱스가 1, 5, 10이면 2,3,4, 6,7,8,9는 결측치가 아니게되므로 2,3,4에는 np.full((1, 5-1-1),1)인
# (1,1,1)을 대입시킨다.
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
# 마지막 record에 대해서는 df의 총 길이에서 결측치의 마지막 값을 뺀 뒤에 1을 빼준 값의 길이에 movie_id값으로 이루어진 
# np 배열을 넣어주었다.
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

# 출력 값
# Movie numpy: [1.000e+00 1.000e+00 1.000e+00 ... 4.499e+03 4.499e+03 4.499e+03]
# Length: 24053764
# Movie numpy의 배열 형태를 보면 4498개의 결측치가 있나보다.. 라고 생각할 수 있다.


# remove those Movie ID rows
# df의 'Rating'열의 값이 결측치가 아닌 모든 열을 다시 df에 대입
df = df[pd.notnull(df['Rating'])]

# df에 'Movie_Id'라는 열을 만들어서 거기에 movie_np를 넣어주는데, 그 전에 movie_np의 값들을 모두 정수형으로 바꾸어서 
# 넣어준다
df['Movie_Id'] = movie_np.astype(int)
# Cust_Id에 대해서도 똑같다.
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.iloc[::5000000, :])

# 출력 값
# -Dataset examples-
#           Cust_Id  Rating  Movie_Id
# 1         1488844     3.0         1
# 5000996    501954     2.0       996
# 10001962   404654     5.0      1962
# 15002876   886608     2.0      2876
# 20003825  1193835     2.0      3825

### Data slicing

The data set now is super huge. I have tried many different ways but can't get the Kernel running as intended without memory error. Therefore I tried to reduce the data volumn by improving the data quality below:
데이터 셋은 매우 크다. 여러 방법을 시도했지만 메모리 에러 없이는 커널을 실행할 수 없었다. 따라서 데이터 품질을 개선해서 데이터의 볼륨을 줄이고자 하였다.

- Remove movie with too less reviews (they are relatively not popular)
- 리뷰가 너무 적은 영화는 제거하였다(상대적으로 덜 인기있는 영화들).
- Remove customer who give too less reviews (they are relatively less active)
- 리뷰를 너무 적게 쓴 고객을 지웠다(상대적으로 덜 활동적인 고객들).

Having above benchmark will have significant improvement on efficiency, since those unpopular movies and non-active customers still occupy same volumn as those popular movies and active customers in the view of matrix (NaN still occupy space). This should help improve the statistical signifiance too.
위의 기준에 따라 개선하였을 때 효율이 크게 증가하였다. 인기가 없는 영화와 비활동적인 고객들은 인기 있는 영화와 활동적인 고객과 행렬의 관점에서는 동일한 비중을 차지하고 있기 때문이었다(여전히 결측치는 존재한다.) 이는 통계적 개선에 큰 도움을 줄 것이다.

Let's see how it is implemented:
어떻게 작동하는지 알아보자:

In [ ]:
f = ['count', 'mean']

# df를 Movie_Id를 기준으로 중복된 값을 묶은 다음에 'Rating'열에 대해서 count와 mean을 수행을 수행하여 
# 각각 count열과 mean열에 넣는다.
# df_movie_summary의 열은 count와 mean열만 추가되었을 것임
df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
# map함수로 df_moive_summary의 index('Movie_Id')를 int형식으로 바꾸어주었음.
df_movie_summary.index = df_movie_summary.index.map(int)
# 'count'열의 백분위 수로 70%에 있는 값을 소수점 1째자리에서 반올림한다. (혹시 소수점이 있을까봐 지우는 듯함)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
# 'count'열의 값이 백분위 수 70%보다 아래에 있는 MovieId는 제외한 나머지 MovieId만 drop_movie_list에 넣는다.
# (인덱스 열) => relatively less popular movie..
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))
# 출력 결과 : 영화 리뷰 개수의 상위 70%에 해당하는 값이 1799개이다. (생각보다 적네..? 아 많나..? 모르겠다 ㅎㅎ;)
# Movie minimum times of review: 1799.0

# 이제 customer에 대해 슬라이싱할 차례
# df의 'Cust_Id'열에 대해서 중복되는 값을 없앤 뒤 'Rating'에 대해 count와 mean을 계산하여 count, mean 열에 넣는다.
# df_cust_summary의 열은 현재 각 Cust_Id별로 계산된 'Rating'의 count값과 mean값이 각각의 열에 들어가있음
df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
# Movie_Id와 똑같이 Cust_Id를 Int형식으로 바꿔준다.
df_cust_summary.index = df_cust_summary.index.map(int)
# count개수의 상위 70퍼센트에 해당하는 값을 찾아 소수점 1째 자리에서 반올림해준다.
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
# 상위 70퍼센트에 해당하는 값보다 적은 리뷰의 개수를 단 고객의 아이디는 삭제한다.(relatively less active customer)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))
# 출력 결과 : 상위 70퍼센트에 해당하는 고객의 리뷰의 개수 : 52개(활동적인 사람 중에서 젤 덜 활동적인..)
# 52개는 좀 많은데..?
# Customer minimum times of review: 52.0

# 기존의 df와 슬라이싱한 df의 volumn차이를 비교하기 위해 .shape메서드로 출력해서 비교해보자.
print('Original Shape: {}'.format(df.shape))
# ~표시로 df의 'Movie_Id'열의 값이 제거된 리스트에 없는 값들만 df에 넣는다.
# ~는 NOT표시
df = df[~df['Movie_Id'].isin(drop_movie_list)]
# 비인기 영화를 제거한 df에 다시 비활동적인 고객의 리스트에 포함되지 않은 고객들만 넣는다.
df = df[~df['Cust_Id'].isin(drop_cust_list)]


print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])

# 출력 결과
# 기존의 개수는 24053764개로 약 2400만개
# 슬라이싱한 이후의 개수는 17337458개로 약 1730만개
# df.iloc[::5000000, :]을 출력했음에도 불구하고 -Data Examples-에 696으로 시작하는 이유는 1부터 695번째 까지의 인덱스에
# 해당하는 값들은 이미 슬라이싱 되었고 696번째 열이 첫 번째 열이기 때문 
# Original Shape: (24053764, 3)
# After Trim Shape: (17337458, 3)
# -Data Examples-
#           Cust_Id  Rating  Movie_Id
# 696        712664     5.0         3
# 6932490   1299309     5.0      1384
# 13860273   400155     3.0      2660
# 20766530   466962     4.0      3923

# Let's pivot the data set and put it into a giant matrix - we need it for our recommendation system:
# 데이터 셋을 pivot하여 큰 메트릭스에 넣자! - 추천시스템을 구현하기 위해 필요하다.
# pivot의 의미 : 데이터 테이블을 재배치한다는 의미
# ex) df1.pivot(index = None, columns = None, values = None)
# index : index로 사용될 column
# columns = column으로 사용될 column
# values = value에 채우고자 하는 column

# df에 대하여 value값에는 'Rating'열을 채우고, 인덱스는 고객의 아이디, column에는 Movie_Id를 넣어준다.
#         Movie_Id ~ ~ ~
# Cust_Id   3.0     3.0 
#    .      4.0     1.0
#    .      3.0     5.0
#    .
# 위와 같은 형식으로 피벗테이블을 만들어준다.
df_p = pd.pivot_table(df,values='Rating',index='Cust_Id',columns='Movie_Id')

print(df_p.shape)
# 출력 결과 : 143458개의 Cust_Id, 1350개의 Movie_Id가 있음
# (143458, 1350)

# --------------------------------------------------------------------------------------------
# Kaggle에서 알려준 데이터프레임을 나누는 다른 방법, 더 나은 방법은 아니라고 함.
# Below is another way I used to sparse the dataframe...doesn't seem to work better

# df의 'Cust_Id'중에 unique한 값들만 가져와서 정렬한다음에 리스트로 만들어 Cust_Id_u에 넣는다.
Cust_Id_u = list(sorted(df['Cust_Id'].unique()))
# df의 'Movie_Id'중에 unique한 값들만 가져와서 정렬한다음에 리스트로 만들어 Movie_Id_u에 넣는다.
Movie_Id_u = list(sorted(df['Movie_Id'].unique()))
# df의 'Rating'열에 해당하는 값들을 리스트로 만들어 data에 넣는다.
data = df['Rating'].tolist()
# df의 'Cust_Id'열의 값을 category형식으로 바꾼 다음 categories에 넣어준 Cust_Id_u에 따라 
# .cat.codes로 임의의 숫자형으로 변경하여 row에 넣어준다.
row = df['Cust_Id'].astype('category', categories=Cust_Id_u).cat.codes
# 위와 동일
col = df['Movie_Id'].astype('category', categories=Movie_Id_u).cat.codes

# Cust_Id_u by Movie_Id_u sparse_matrix(희소행렬: 비교적 0이 많은 행렬)를 만들어준다.
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(Cust_Id_u), len(Movie_Id_u)))
# why convert the sparse matrix to dense matrix ?
df_p = pd.DataFrame(sparse_matrix.todense(), index=Cust_Id_u, columns=Movie_Id_u)
# 0을 결측치로 바꿔준다.
df_p = df_p.replace(0, np.NaN)
# --------------------------------------------------------------------------------------------

### Data mapping
Now we load the movie mapping file:

In [ ]:
# csv파일을 읽어온 뒤 column 이름을 'Movie_Id', 'Year', 'Name'으로 가져온다.
df_title = pd.read_csv('../input/movie_titles.csv', encoding = "ISO-8859-1", 
header = None, names = ['Movie_Id', 'Year', 'Name'])

# index는 'Movie_Id'로 설정한다.
# inplace = True를 넣어주면 df_title = df_title.set_index('Movie_Id')처럼 다시 넣어줄 필요 없이 제자리에서
# 수정해준다. 와 이거 혁명이다.
df_title.set_index('Movie_Id', inplace = True)
print (df_title.head(10))
# 출력 값
#             Year                          Name
# Movie_Id                                      
# 1         2003.0               Dinosaur Planet
# 2         2004.0    Isle of Man TT 2004 Review
# 3         1997.0                     Character
# 4         1994.0  Paula Abdul's Get Up & Dance
# 5         2004.0      The Rise and Fall of ECW
# 6         1997.0                          Sick
# 7         1992.0                         8 Man
# 8         2004.0    What the #$*! Do We Know!?
# 9         1991.0      Class of Nuke 'Em High 2
# 10        2001.0                       Fighter

### Recommendation models
Well all data required is loaded and cleaned! Next let's get into the recommendation system.

### Recommend with Collaborative Filtering
Evalute performance of collaborative filtering, with just first 100K rows for faster process:

아직 CF에 대해 공부를 덜 했습니다..!